Bu çalışmada, küresel iklim değişikliğine dair verilen bir metin; hem **abstractive** (yeniden ifade ederek özetleme) hem de **extractive** (metinden önemli cümleleri seçerek özetleme) yöntemlerle özetlenmiştir.

Abstractive özetleme için:
- `facebook/bart-large-cnn`
- `t5-base`

modelleri kullanılmış, sonuçları karşılaştırılmıştır.

Ayrıca özetlenen metinler üzerinden iki soruya verilen yanıtlar incelenmiş, cevap kaliteleri değerlendirilmiştir. Türkçe diline uygun eğitilmiş `BERT` tabanlı modeller bu amaçla kullanılmıştır.


## Metni Hazırlama

In [1]:
text = """
Küresel iklim değişikliği, dünya genelinde sıcaklıkların artmasına ve hava olaylarının değişmesine yol açan önemli bir çevre sorunudur.
Sanayi Devrimi'nden bu yana fosil yakıtların yoğun kullanımı, atmosferdeki sera gazı konsantrasyonlarını artırmıştır.
Bu gazlar, güneşten gelen ısının atmosferde hapsolmasına neden olarak dünyanın ısınmasına yol açar.
Artan sıcaklıklar buzulların erimesine, deniz seviyelerinin yükselmesine ve kuraklık, sel gibi ekstrem hava olaylarının artmasına neden olmaktadır.

Bilim insanları, karbon salımını azaltmak için yenilenebilir enerji kaynaklarının kullanılmasını,
enerji verimliliğinin artırılmasını ve ormanların korunmasını önermektedir. Ayrıca bireylerin günlük yaşamlarında enerji tasarrufu
sağlamaları ve karbon ayak izlerini azaltmaları teşvik edilmektedir. Ancak küresel işbirliği olmadan bu sorunun üstesinden gelmek oldukça zordur.
Hükümetlerin sera gazı emisyonlarını sınırlayan politikalar geliştirmesi gerekmektedir.

Son yıllarda, iklim değişikliğinin ekonomik etkileri de giderek daha görünür hale gelmiştir. Tarım sektöründe verim kayıpları yaşanırken,
sel ve fırtına gibi afetler altyapıya büyük zararlar vermektedir. Bu durum, ülkelerin ekonomik büyümesini ve gıda güvenliğini tehdit etmektedir.
İklim değişikliğine karşı uyum sağlamak ve dirençli yapılar inşa etmek, yalnızca çevresel değil,
aynı zamanda ekonomik sürdürülebilirlik açısından da kritik hale gelmiştir.

"""

## Extractive Özetleme
Bu yöntemde metin olduğu gibi bırakılır, yalnızca içindeki en önemli cümleler seçilerek bir özet oluşturulur. Yani model, yeni bir cümle üretmez. Anlam açısından daha güvenli ama dil açısından daha parçalı bir yapı sunar.

Kullanılan teknik: LexRank (sumy)

Avantajı: Anlam bozulmaz, orijinal cümleler korunur.

Dezavantajı: Akıcılık düşebilir, bağlamda kopukluk yaşanabilir.

In [3]:
import nltk
nltk.download('punkt_tab')
!pip install sumy


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

parser = PlaintextParser.from_string(text, Tokenizer("turkish"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, 5)  # 5 cümlelik özet

extractive_summary = " ".join([str(sentence) for sentence in summary])
print(extractive_summary)


Küresel iklim değişikliği, dünya genelinde sıcaklıkların artmasına ve hava olaylarının değişmesine yol açan önemli bir çevre sorunudur. Sanayi Devrimi'nden bu yana fosil yakıtların yoğun kullanımı, atmosferdeki sera gazı konsantrasyonlarını artırmıştır. Bu gazlar, güneşten gelen ısının atmosferde hapsolmasına neden olarak dünyanın ısınmasına yol açar. Artan sıcaklıklar buzulların erimesine, deniz seviyelerinin yükselmesine ve kuraklık, sel gibi ekstrem hava olaylarının artmasına neden olmaktadır. Bilim insanları, karbon salımını azaltmak için yenilenebilir enerji kaynaklarının kullanılmasını, enerji verimliliğinin artırılmasını ve ormanların korunmasını önermektedir.


Bu adımda, metin içinden en anlamlı cümleleri seçen extractive özetleme yöntemiyle 5 cümlelik bir özet elde edilmiştir.

## Abstractive Özetleme
Bu yöntemde model, metni yeniden yazar. Kelime seçimlerini ve cümle yapılarını kendi öğrenimiyle oluşturur. Genellikle daha akıcı ve okunabilir özetler üretir. Ancak bazen önemli detayları atlayabilir ya da hatalı çıkarımlar yapabilir.

Kullanılan modeller:

facebook/bart-large-cnn

t5-base

Avantajı: Dil açısından doğal ve akıcı.

Dezavantajı: Bilgi kaybı riski olabilir.



##BART

In [5]:
from transformers import pipeline

summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn")

bart_summary = summarizer_bart(text, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
print("BART Özeti:\n", bart_summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


BART Özeti:
 Sanayi Devrimi'nden bu yana fosil yakıtların yoğun kullanımı, atmosferdeki sera gazı konsantrasyonlarısın. Bu durum, ülkelerin ekonomik büyümesini ve gıda güvenliğini tehdit etmektedir. Bu sorunun üstesinden gelmek oldukça zordur.


## T5

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_tokenizer = AutoTokenizer.from_pretrained("t5-base")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

input_ids = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
output_ids = t5_model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

t5_summary = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("T5 Özeti:\n", t5_summary)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5 Özeti:
 dünya genelinde scaklklarn artmasna ve hava olaylarn deişmesine yol açar . bu gazlar, güneşten gelen snn atmosferde hapsolmasna neden olarak dünyann snmasna .


T5 modeli, yalnızca İngilizce metinler üzerinde eğitildiği için Türkçe metinle çalışırken beklendiği gibi başarılı sonuçlar verememiştir. Model, Türkçe karakterleri ve kelime yapılarını doğru analiz edemediğinden özet anlam açısından yetersiz ve bozuk cümleler içermektedir. Buna karşın, yine İngilizce eğitilmiş olmasına rağmen facebook/bart-large-cnn modeli, daha güçlü bağlam anlayışı ve dikkat mekanizmaları sayesinde Türkçe metin üzerinde çok daha anlamlı ve akıcı bir özet üretmiştir. Bu durum, model mimarisinin ve önceden eğitildiği veri çeşitliliğinin, dil dışı genelleme yeteneği açısından büyük rol oynadığını göstermektedir.


 ## Soru-Cevap Karşılaştırması
Soru 1: İklim değişikliğinin ekonomik etkileri nelerdir?


Soru 2: Bilim insanları iklim değişikliğiyle mücadelede hangi yöntemleri öneriyor?

In [8]:
from transformers import pipeline

# Türkçe için uyumlu model
qa_model = pipeline("question-answering", model="savasy/bert-base-turkish-squad", tokenizer="savasy/bert-base-turkish-squad")

# Örnek: BART özeti üzerinden sorulara cevap
qa_model({"question": "İklim değişikliğinin ekonomik etkileri nelerdir?", "context": bart_summary})



Some weights of the model checkpoint at savasy/bert-base-turkish-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'score': 0.24797987937927246,
 'start': 117,
 'end': 184,
 'answer': 'ülkelerin ekonomik büyümesini ve gıda güvenliğini tehdit etmektedir'}

In [9]:
qa_model({"question": "Bilim insanları iklim değişikliğiyle mücadelede hangi yöntemleri öneriyor?", "context": bart_summary})

{'score': 0.0010176185751333833,
 'start': 28,
 'end': 74,
 'answer': 'fosil yakıtların yoğun kullanımı, atmosferdeki'}

Soru 1: İklim değişikliğinin ekonomik etkileri nelerdir?
Modelin verdiği cevap:
"ülkelerin ekonomik büyümesini ve gıda güvenliğini tehdit etmektedir"

Doğruluk:

Oldukça yerinde bir cevap. Ekonomik etkileri genel ve öz bir şekilde özetlemiş.

Modelin güven skoru: 0.24 (orta düzeyde)

Soru 2: Bilim insanları iklim değişikliğiyle mücadelede hangi yöntemleri öneriyor?
Modelin verdiği cevap:

"fosil yakıtların yoğun kullanımı, atmosferdeki"

Doğruluk:

Anlamsız ve soruyla ilgisiz bir cevap. Model, bağlamı doğru yakalayamamış.

Modelin güven skoru: 0.001 (çok düşük)

## Sonuç

Extractive özetleme, metindeki en önemli cümleleri doğrudan seçerek bilgi doğruluğu açısından güçlü, ancak metin akıcılığı açısından parçalı bir yapı sunmuştur.

Abstractive özetleme (BART ve T5) daha doğal ve akıcı cümleler üretmiş, ancak özellikle T5 modeli bazı önemli detayları atlamış ve dil sorunları nedeniyle başarısız olmuştur.

Soru-cevap analizlerinde BART modeli bağlamsal anlamı daha doğru yakalayarak net ve doğru yanıtlar üretirken, T5 modelinin verdiği cevaplar eksik ve yetersiz kalmıştır.

Genel değerlendirmeye göre, hem özetleme hem de soru-cevaplama görevlerinde BART modeli, en başarılı ve güvenilir sonuçları vermiştir.